# RAG + Image Generator Agent(보고서 작성)

이번 튜토리얼에서는 웹 검색(Web Search), PDF 문서 기반 검색(RAG), 이미지 생성(Image Generation) 등을 통해 보고서를 작성하는 에이전트를 만들어 보겠습니다.

**참고**

- [Agent Toolkits](https://api.python.langchain.com/en/latest/community/agent_toolkits.html)

- [Tools](https://python.langchain.com/docs/integrations/tools/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agent-Projects")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agent-Projects


## 도구 정의

### 웹 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

`.env` 파일에 다음과 같이 등록합니다.

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 문서 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.

**실습에 활용한 문서**

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


다음은 retriever 를 생성하고, 생성한 retriever 를 기반으로 도구를 생성합니다.

먼저, 문서를 로드하고, 분할한 뒤 retriever 를 생성합니다.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

다음은 retriever 를 도구로 정의합니다.

이때, `document_prompt` 는 문서의 내용을 표시하는 템플릿을 정의합니다.

**참고**

- 기본 값은 문서의 `page_content` 만 표기합니다. 

- 따라서, 나중에 문서의 페이지 번호나 출처등을 표시하기 위해서는 템플릿을 따로 정의해야 합니다.

In [8]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

# 문서의 내용을 표시하는 템플릿을 정의합니다.
document_prompt = PromptTemplate.from_template(
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

# retriever 를 도구로 정의합니다.
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search for information in the PDF file",
    document_prompt=document_prompt,
)

### DallE 이미지 생성 도구

이번에는 Dall-E 이미지 생성 도구를 생성합니다.

**주요 속성**

- `model`: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")

- `n`: 생성할 이미지 수 (기본값: 1)

- `size`: 생성할 이미지 크기
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style`: 생성될 이미지의 스타일 (기본값: "natural", "vivid")

- `quality`: 생성될 이미지의 품질 (기본값: "standard", "hd")

- `max_retries`: 생성 시 최대 재시도 횟수

In [11]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.tools import tool

# DallE API Wrapper를 생성합니다.
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)


# DallE API Wrapper를 도구로 정의합니다.
@tool
def dalle_tool(query):
    """use this tool to generate image from text"""
    return dalle.run(query)

### 파일 관리 도구

**파일 관리 도구들**

- `CopyFileTool`: 파일 복사
  
- `DeleteFileTool`: 파일 삭제

- `FileSearchTool`: 파일 검색

- `MoveFileTool`: 파일 이동

- `ReadFileTool`: 파일 읽기

- `WriteFileTool`: 파일 쓰기

- `ListDirectoryTool`: 디렉토리 목록 조회

In [13]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 작업 디렉토리 경로 설정
working_directory = "tmp"

# 파일 관리 도구 생성(파일 쓰기, 읽기, 디렉토리 목록 조회)
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

# 생성된 파일 관리 도구 출력
file_tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

자, 이제 모든 도구를 종합합니다.

In [14]:
tools = file_tools + [
    retriever_tool,
    search,
    dalle_tool,
]

# 최종 도구 목록 출력
tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 Tool(name='pdf_search', description='use this tool to search for information in the PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x133b47d80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x30f560610>), document_prompt=PromptTemplate(input_variables=['page', 'page_content', 'source'], template='<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x133b47e20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x30f560610>), document_prompt=PromptTemplate(input_variables=['page',

## Agent 생성

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import AgentStreamParser

# session_id 를 저장할 딕셔너리 생성
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "You are a professional researcher. "
            "You can use the pdf_search tool to search for information in the PDF file. "
            "You can find further information by using search tool. "
            "You can use image generation tool to generate image from text. "
            "Finally, you can use file management tool to save your research result into files.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

# 스트림 파서 생성
agent_stream_parser = AgentStreamParser()

자, 이제 에이전트를 실행해 봅시다.

In [17]:
# 에이전트 실행
result = agent_with_chat_history.stream(
    {
        "input": "삼성전자가 개발한 `생성형 AI` 와 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: <document><content>SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창

생성된 보고서 파일(`report.md`)의 내용을 확인하면 다음과 같이 출력됩니다.

![](./assets/report-agent-01.png)

다음으로는 웹 검색을 통해 보고서 파일을 업데이트 해 봅시다.

In [18]:
# 웹 검색을 통해 보고서 파일 업데이트
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 삼성전자가 개발한 `생성형 AI` 와 관련된 정보들을 웹 검색하고, 검색한 결과를 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: tavily_search_results_json
query: 삼성전자 생성형 AI
Log: 
Invoking: `tavily_search_results_json` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: [{'url': 'https://www.yna.co.kr/view/AKR20231108041400003', 'content': "삼성전자[005930]가 자체 개발한 생성형 인공지능(AI) 모델 '삼성 가우스'(Samsung Gauss)가 처음 공개됐다. 삼성전자는 삼성 가우스를 활용해 임직원의 업무 생산성을 높이는 한편, 생성형 AI 모델을 단계적으로 제품에 탑재해 새로운 사용자 경험을 제공한다는 계획이다."}, {'url': 'https://www.aitimes.kr/news/articleView.html?idxno=29318', 'content': "삼성전자가 8일, 삼성전자 서울R&D캠퍼스에서 열린 '삼성 AI 포럼 2023'을 통해 생성 AI 모델 '삼성 가우스(Samsung Gauss)'를 최초 공개했다. 삼성전자는 '삼성 가우스'를 활용해 회사 내 업무 혁신을 추진하고 나아가 사람들의 일상에 새로운 경험을 제공하기 위해 생성형 AI 기술을 발전시킨다는 계획이다 ..."}, {'url': 'https://zdnet.co.kr/view/?no=20231108081251', 'content': "삼성전자가 자체 개발한 생성형 AI 모델 '삼성 가우스(Samsung Gauss)'를 최초로 공개했다. 삼성전자는 가우스를 활용해 회사 내 업무 혁신을 추진하고 ..."}, {'url': 'https://namu.wiki/w/삼성+가우스', 'content': '삼성 가우스(Samsung Gauss)는 삼성전자의 생성형 인공지능 모델이다. 클라우드와 온디바이스를 위한 여러 생성형 AI 모델이 묶인 초거대 생성형 AI모델 패밀리로, 생성형 AI 모델과 어시스

업데이트된 보고서 파일(`report.md`)의 내용을 확인하면 다음과 같이 출력됩니다.

![](./assets/report-agent-02.png)

In [19]:
# 보고서 작성을 요청합니다.
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report.md
Log: 
Invoking: `read_file` with `{'file_path': 'report.md'}`



[관찰 내용]
Observation: ## 삼성전자의 생성형 AI ‘삼성 가우스’ 관련 정보

- **페이지 번호**: 10, 12  
- **파일명**: SPRI_AI_Brief_2023년12월호_F.pdf  

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 ‘삼성 가우스’를 공개함.  
- 삼성 가우스는 언어, 코드, 이미지의 3개 모델로 구성됨.  
- 온디바이스 작동으로 사용자 정보 유출 위험이 없음을 특징으로 함.  
- 삼성 가우스는 다양한 제품에 단계적으로 탑재될 예정임.  
- 언어 모델은 메일 작성, 문서 요약, 번역 등을 지원하며, 클라우드와 온디바이스 모델이 있음.  
- 코드 모델인 ‘코드아이(code.i)’는 대화형 인터페이스로 소프트웨어 개발에 최적화됨.  
- 이미지 모델은 창의적인 이미지 생성 및 저해상도 이미지를 고해상도로 변환하는 기능을 제공함.  
- 2024년부터 가우스 탑재 삼성 스마트폰이 경쟁력을 가질 것으로 기대됨.## 삼성전자의 생성형 AI ‘삼성 가우스’ 관련 웹 검색 정보

- **출처**: [연합뉴스](https://www.yna.co.kr/view/AKR20231108041400003)
- 삼성전자가 개발한 생성형 AI 모델 '삼성 가우스'가 처음 공개됨.
- 삼성전자는 이 모델을 통해 임직원의 업무 생산성을 높이고, 생성형 AI 모델을 제품에 단계적으로 탑재하여 새로운 사용자 경험을 제공할 계획임.

- **출처**: [AI타임스](https://www.aitimes.kr/news/articleView.html?idxno=29318)
- 삼성전자는 '삼성 AI 포럼 2023'에서 생성형 AI 모델 '삼성 가우스'를 최초 공개함.
- 이 모델은 회사 내 업무 혁신을 추진하고, 사람들의

새롭게 작성된 보고서 파일(`report-2.md`)의 내용을 확인하면 다음과 같이 출력됩니다.

![](./assets/report-agent-03.png)

마지막으로, 보고서 내용을 기반으로 이미지 생성을 요청해 봅시다.

In [20]:
# 이미지 생성을 요청합니다.
result = agent_with_chat_history.stream(
    {
        "input": "`report-2.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용에 어울리는 이미지를 생성하세요. "
        "생성한 이미지의 url 을 markdown 형식으로 보고서의 가장 상단에 추가하세요. "
        "마지막으로 작성된 결과물을 `report-3.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:
[도구 호출]
Tool: read_file
file_path: report-2.md
Log: 
Invoking: `read_file` with `{'file_path': 'report-2.md'}`



[관찰 내용]
Observation: # 삼성전자의 생성형 AI ‘삼성 가우스’ 보고서

## 1. 개요
삼성전자가 개발한 생성형 AI 모델 '삼성 가우스'는 온디바이스에서 작동하며, 사용자 정보 유출 위험을 최소화하는 특징을 갖고 있다. 이 모델은 언어, 코드, 이미지를 포함하는 세 가지 주요 모델로 구성되어 있으며, 다양한 삼성 제품에 단계적으로 탑재될 예정이다. 본 보고서는 삼성 가우스의 핵심 내용과 기대 효과를 살펴본다.

## 2. 핵심내용  
| 항목 | 내용 |
|------|------|
| 모델명 | 삼성 가우스 |
| 구성 모델 | 언어 모델, 코드 모델, 이미지 모델 |
| 특징 | 온디바이스 작동으로 정보 유출 위험 최소화 |
| 언어 모델 기능 | 메일 작성, 문서 요약, 번역 등 |
| 코드 모델 | 코드아이(code.i) - 대화형 인터페이스 제공 |
| 이미지 모델 기능 | 창의적 이미지 생성, 저해상도 이미지 고해상도로 변환 |
| 출시 예정 | 2024년부터 삼성 스마트폰에 탑재 예정 |
| 주요 목표 | 임직원 업무 생산성 향상, 새로운 사용자 경험 제공 |

## 3. 최종결론
삼성전자의 생성형 AI '삼성 가우스'는 회사의 업무 혁신을 위한 중요한 기술로, 다양한 분야에 적용 가능성을 지닌 모델이다. 이 모델은 사용자 정보를 안전하게 보호하면서도 다양한 기능을 제공하여, 향후 사용자 경험을 혁신적으로 변화시킬 것으로 기대된다.  

**출처**: SPRI_AI_Brief_2023년12월호_F.pdf, [연합뉴스](https://www.yna.co.kr/view/AKR20231108041400003), [AI타임스](https://www.aitimes.kr/news/articleView.html?idx

마지막으로 생성된 보고서 파일(`report-3.md`)의 내용을 일부를 확인하면 다음과 같이 출력됩니다.

![](./assets/report-agent-04.png)